# RUL Prediction Video

In [1]:
import numpy as np
import torch
from pickle import load,dump
from datetime import date
import os

# Custom modules
from method import compute_eval_matrices
from visualization import make_rul_video 
from watercan import GroupWaterCan
from helpers import get_from, np2gpu

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Fix Seed

In [3]:
seed=0
# Set seed for NumPy
np.random.seed(seed)

import torch

# Set seed for PyTorch (both CPU and CUDA if available)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Get the data

In [4]:
dataset_name='N-CMAPSS_DS05' #'uav_data'#
threshold_name=''#'-0.1' #  '+0.1' #

#Eval parameters
std_bound=0
poly_deg=0

# Load test data

In [5]:
with open(f'./data/{dataset_name}/performs_info.pkl', 'rb') as file:
    performs_info=load(file)

with open(f'./data/{dataset_name}/time_info.pkl', 'rb') as file:
    time_info=load(file)
    
with open(f'./data/{dataset_name}/times_test{threshold_name}.pkl', 'rb') as file:
    t_observ = load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}_threshold.pkl', 'rb') as file:
    times_thresholds = load(file)

with open(f'./data/{dataset_name}/RUL_test{threshold_name}.pkl', 'rb') as file:
    rul= load(file)

with open(f'./data/{dataset_name}/EOLcause_test{threshold_name}.pkl', 'rb') as file:
    EOLcause= load(file)

performs = {}
thresholds = {}
for name in performs_info.keys():
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}.pkl', 'rb') as file:
        performs[name] = load(file)
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}_threshold.pkl', 'rb') as file:
        thresholds[name]= load(file)

EOLcause

['HPC_eff', 'HPC_eff', 'HPC_eff', 'HPC_eff']

## Add time 0

In [6]:
rul_intercept=[r[0]-(r[0]-r[1])/(time[0]-time[1])*time[0] for time,r in zip(t_observ,rul)]
rul=[np.concatenate([np.array((inter,)),ru])  for inter,ru in zip(rul_intercept,rul)]

## Choose Library

In [7]:
library_address=f'./Library/MTL_2024-09-23_{dataset_name}/'

## Load Group Distribution

In [8]:
group_distributions={}
for perform_name in performs_info.keys():
    perform_info=performs_info[perform_name]

    perform_address=library_address+f'{perform_name}/'

    configs=torch.load(perform_address+'config.pth')
    model_state=torch.load(perform_address+'model_state.pth')
    model=GroupWaterCan(configs)
    model.load_state_dict(model_state)
    model.to(device)
    if poly_deg or std_bound:
        with open(f'./data/{dataset_name}/times.pkl', 'rb') as file:
            train_time = load(file)
        xs_train=[np2gpu(time,device).unsqueeze(1) for time in train_time]
        if std_bound:
            max_std_ontrain=max([max(out[:,1]) for out in model(xs_train)]).item()
            model.std_bound=std_bound*max_std_ontrain
        if poly_deg:
            model.set_taylor_extension(xs_train,poly_deg)
            model.extend()
    group_distributions[perform_name]=model
t_observ_torch=[torch.tensor(time,device=device,dtype=torch.float32).unsqueeze(dim=1) for time in t_observ]

# Compute and prepare A,b matrices and time

In [9]:
n_test_unit=len(t_observ)
As,Ss=compute_eval_matrices(group_distributions,t_observ_torch)
test_unit_list=range(n_test_unit)
bs = performs

## Choose Experiment parameters

In [10]:
# Performance-independent parameters
n_train_unit =6 #20 # 
n_paths = 60 # 20 #
lr=1
forgetting=1
conserv_pred=0.5
gama_value=1

# Performance-dependent parameters
gamma={key:gama_value for key,perform in performs_info.items()} #perform['default_gamma'] #reaction_delay
max_life =time_info['max_life']


# Visualization
accel=15*3600
time_unit=time_info['time_unit']
#max_life=max([time[-1] for time in t_observ])
t = np.linspace(0, int(max_life+5), int((max_life+5)*20))

In [11]:
config={'n_train_unit':n_train_unit,
        'n_paths':n_paths,
        'lr':lr,
        'conserv_pred':conserv_pred,
        'gamma':gamma,
        'max_life':max_life}

## Set Experiment name

In [12]:
today = date.today()
threshold_text=0 if not threshold_name else threshold_name

In [13]:
exp=f'threshold{threshold_text}_{today}_paper_gamma{gama_value}_lr{lr}_forgetting{forgetting}_seed{seed}/'

#exp='try'

if exp == 'try':
    n_train_unit =5 #5#
    n_paths =5 #5# 

## Save experiment configuration

In [14]:
exp_address=library_address+f'_Experiments/{exp}'
os.makedirs(exp_address, exist_ok=True)
with open(exp_address+'config.pth', "wb") as f:
    dump(config, f)

## Predicts

In [15]:
## Setup Method
y_lim=get_from(performs_info,'y_lim')
loc=get_from(performs_info,'loc')
monot=get_from(performs_info,'monot')
EOL_cause_text =get_from(performs_info,'EOL_cause') 

subgroup_distributions={name:group_dist.get_sub_watercan(range(n_train_unit)) for name,group_dist in group_distributions.items()}

## Results initialization
trues = []
pred_names='acc','csv','unc'
RMSEs={key: [] for key in pred_names}
predss={key: [] for key in pred_names}
causess={key: [] for key in pred_names}

# For inmidiate information
pred_acc=[]
true=[]


n_unit=1 if exp=='try' else n_test_unit
for unit in range(n_unit): 
        A={}
        S={}
        b={}
        threshold={}
        for name in performs_info.keys():
                A[name]= As[name][unit][:,:n_train_unit]
                S[name]= Ss[name][unit][:,:n_train_unit]
                b[name]= bs[name][unit]
                thres=np.interp(t, times_thresholds[unit], thresholds[name][unit])
                threshold[name]=torch.tensor(thres,dtype=torch.float32,device=device)

        preds,causes=make_rul_video(unit,t,t_observ[unit],rul[unit],EOLcause[unit],
                                monot, threshold, subgroup_distributions, A, b, S, gamma, y_lim, loc,
                                perform_names=performs_info.keys(),causes_text=EOL_cause_text,time_unit=time_unit,
                                n_train_unit=n_train_unit, n_paths=n_paths, lr=lr,forgetting=forgetting,max_life = max_life,
                                accel=accel//3600, save=exp_address,conserv=conserv_pred)
        y_true=rul[unit]

        for name in pred_names:
                RMSE=np.sqrt(np.mean((preds[name]-y_true)**2))
                print(f"RMSE_{name}: {RMSE}")
                RMSEs[name].append(RMSE)
                predss[name].append(preds[name])
                causess[name].append(causes[name])
        trues.append(y_true)


## Save predictions
torch.save(predss,f'{exp_address}'+'preds.pt')
torch.save(causess,f'{exp_address}'+'causes.pt')
torch.save(trues,f'{exp_address}'+'trues.pt')


## Accurate prediction Results
print('----------------------------------')
error=np.concatenate(predss['acc'])-np.concatenate(trues)
RMSE=np.sqrt(np.mean(error**2))
print(f"RMSE_{name} mean: {RMSE}")
print(f"RMSE_{name} std: {np.std(error)}")
print('----------------------------------')


time 0, present alpha var: [0.14124294 0.0960452  0.15254237 0.18644068 0.15254237 0.11864407]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 0, present alpha var: [0.15254237 0.08474576 0.19774011 0.1299435  0.11864407 0.16384181]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 1, present alpha var: [0.12833335 0.29705873 0.08859163 0.05981674 0.12904365 0.9821475 ]
time 1, future alpha var: [0.21420036 0.25198084 0.22620609 0.19361984 0.19854107 0.18032476]
time 1, present alpha var: [0.0795254  0.06776305 0.18434002 0.7339124  0.13650079 0.10506715]
time 1, future alpha var: [0.22169151 0.20623065 0.23662026 0.2000786  0.22541834 0.23654347]
time 2, present alpha var: [0.19682905 0.09340719 0.5549142  0.03583113 0.56175846 1.0724479 ]
time 2, future alpha var: [0.25123358 0.29617059 0.541871   0.19094258 0.24852027 1.21934815]
time 2, present alpha var: [0.03032095 0.11728054 0.384

Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-09-23_N-CMAPSS_DS05/_Experiments/threshold0_2024-09-23_paper_gamma1_lr1_forgetting1_seed0/unit0_pred.mp4
RMSE_acc: 10.905513860343287
RMSE_csv: 17.76591981791948
RMSE_unc: 11.81888516365346
time 0, present alpha var: [0.1299435  0.14124294 0.17514124 0.20903955 0.07344633 0.11864407]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 0, present alpha var: [0.1299435  0.11864407 0.11864407 0.16384181 0.20903955 0.10734463]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 1, present alpha var: [0.06788452 0.11477804 0.17228322 0.05201944 0.07462028 0.09186469]
time 1, future alpha var: [0.19645523 0.24314925 0.21919081 0.1944578  0.20885622 0.23899132]
time 1, present alpha var: [0.04231283 0.13072817 0.24287954 1.2949138  0.35823405 0.0811829 ]
time 1, future alpha var: [0.18780433 0.23491436 0.25186239 0.19778779 0.22878268 0.231694

Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-09-23_N-CMAPSS_DS05/_Experiments/threshold0_2024-09-23_paper_gamma1_lr1_forgetting1_seed0/unit1_pred.mp4
RMSE_acc: 11.84914232816213
RMSE_csv: 22.89365775094472
RMSE_unc: 8.78177460524872
time 0, present alpha var: [0.14124294 0.1299435  0.10734463 0.18644068 0.15254237 0.1299435 ]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 0, present alpha var: [0.15254237 0.06214689 0.15254237 0.14124294 0.15254237 0.18644068]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 1, present alpha var: [0.122393   0.15890075 0.0594798  0.10043001 0.06901193 0.8279012 ]
time 1, future alpha var: [0.17596451 0.21696277 0.19433024 0.17518256 0.19989719 0.25846917]
time 1, present alpha var: [0.05126117 0.10459451 0.11993498 0.8298366  0.16176426 0.10234023]
time 1, future alpha var: [0.19274388 0.20771472 0.22550313 0.21004026 0.2381069  0.22262616

Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-09-23_N-CMAPSS_DS05/_Experiments/threshold0_2024-09-23_paper_gamma1_lr1_forgetting1_seed0/unit2_pred.mp4
RMSE_acc: 3.2293298729878046
RMSE_csv: 6.785488718055402
RMSE_unc: 23.18897521298799
time 0, present alpha var: [0.1299435  0.16384181 0.1299435  0.0960452  0.16384181 0.16384181]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 0, present alpha var: [0.11864407 0.15254237 0.11864407 0.1299435  0.1299435  0.19774011]
time 0, future alpha var: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
time 1, present alpha var: [0.28939587 0.14095439 0.10661635 0.13584591 0.14191082 1.2404034 ]
time 1, future alpha var: [0.26001226 0.2751901  0.21491319 0.18591328 0.18852105 0.20837345]
time 1, present alpha var: [0.04646192 0.06507257 0.10716985 0.07823688 0.05622339 0.13438135]
time 1, future alpha var: [0.1888546  0.20333571 0.23502453 0.2146328  0.19283711 0.295407

#### Load for checking

In [21]:
predss=torch.load(f'{exp_address}/preds.pt')
trues=torch.load(f'{exp_address}/trues.pt')

In [22]:
for name in pred_names:
        print(f'------------------------------- {name} RMSE -------------------------------')
        for unit in range(len(trues)):
                RMSE=np.sqrt(np.mean((predss[name][unit]-trues[unit])**2))
                print(f"unit {unit}: {RMSE}")
        print('----------------------------------')
        error=np.concatenate(predss[name])-np.concatenate(trues)
        RMSE=np.sqrt(np.mean(error**2))
        print(f"RMSE_{name} mean: {RMSE}")
        print(f"RMSE_{name} std: {np.std(error)}")
        print('----------------------------------')

------------------------------- acc RMSE -------------------------------
unit 0: 4.14560568694466
unit 1: 8.292217409427794
unit 2: 6.273527146441853
unit 3: 3.897790538797249
----------------------------------
RMSE_acc mean: 5.97578193950546
RMSE_acc std: 5.741863324866999
----------------------------------
------------------------------- csv RMSE -------------------------------
unit 0: 14.538641374738418
unit 1: 19.380907234115522
unit 2: 3.9587155203965554
unit 3: 12.408702736968834
----------------------------------
RMSE_csv mean: 14.186074615302733
RMSE_csv std: 10.133588046842796
----------------------------------
------------------------------- unc RMSE -------------------------------
unit 0: 12.78170931539502
unit 1: 7.550554255497897
unit 2: 22.339906637482365
unit 3: 15.012845905046609
----------------------------------
RMSE_unc mean: 14.84430780872728
RMSE_unc std: 9.095842396475721
----------------------------------


In [23]:
exp_address

'./Library/MTL_2024-08-28_N-CMAPSS_DS05/_Experiments/threshold0_2024-09-22_paper_gamma1_lr1_forgetting1_seed0/'